In [ ]:
!pip install yfinance numpy pandas seabron scikit-learn optuna seaborn xgboost joblib

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import optuna
import warnings
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split
import joblib

c:\Users\alexi\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 10px;
">

<b>Ativo de Teste — Itaú (Mapeamento Inicial de Features)</b><br>
Selecionado como ativo de referência para validação e mapeamento completo das features iniciais do modelo.  
Todos os testes a seguir servirão de base para o **Modelo A**, no qual essas etapas serão automatizadas, incluindo **engenharia de dados** e **tratamento de features**.

<br><br>
<b>I) Dados de Preço e Volume</b>
<ul>
  <li>Preço de Abertura (<i>Open</i>)</li>
  <li>Preço de Fechamento (<i>Close</i>)</li>
  <li>Máxima (<i>High</i>)</li>
  <li>Mínima (<i>Low</i>)</li>
  <li>Volume</li>
  <li>Amplitude Diária (<i>Daily Range</i>)</li>
  <li>Corpo Real (<i>Real Body</i>)</li>
  <li>Sombras Superior e Inferior (<i>Upper / Lower Shadow</i>)</li>
  <li>Log-Retorno (<i>t</i>, <i>t−1</i>)</li>
  <li>Janelas Deslizantes — 14, 16, 30 e 60 períodos</li>
</ul>

<b>II) Indicadores Técnicos</b>
<ul>
  <li>Média Móvel Simples (SMA — 7 e 21)</li>
  <li>Média Móvel Exponencial (EMA — 12 e 26)</li>
  <li>RSI (7 e 14)</li>
  <li>MACD (12, 26 e Sinal)</li>
  <li>Bandas de Bollinger (21 períodos)</li>
  <li>ATR (7 e 14)</li>
  <li>High–Low Spread</li>
  <li>Squeeze Momentum (<i>SQZ</i>)</li>
</ul>

</div>


In [ ]:
DATA_INICIAL = '2019-04-09'
DATA_FIM = '2025-10-01'

In [ ]:
ticker = "ITUB4.SA"
itau = yf.download(ticker, start=DATA_INICIAL, end=DATA_FIM)
print(itau.head())
if isinstance(itau.columns, pd.MultiIndex):
    itau.columns = itau.columns.get_level_values(0)
itau.columns

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Amplitude Diária (<i>Daily Range</i>)</b><br>
Calculada pela diferença aritmética entre a máxima e a mínima do dia (<i>H<sub>t</sub> − L<sub>t</sub></i>).  
Captura a <b>volatilidade intradiária absoluta</b>, servindo como proxy para a <b>incerteza</b> e a <b>agressividade</b> dos participantes do mercado durante o pregão, independentemente da direção do fechamento.

</div>

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Corpo Real (<i>Real Body</i>)</b><br>
Representa o deslocamento líquido do preço, calculado como (<i>C<sub>t</sub> − O<sub>t</sub></i>).  
Quantifica matematicamente a <b>força direcional</b> e a <b>convicção</b> do movimento: valores positivos indicam domínio comprador, enquanto valores negativos indicam domínio vendedor.  
Para o modelo, é essencial para diferenciar dias de <b>alta convicção</b> de períodos de <b>indecisão</b> (ex.: <i>Doji</i>).

</div>

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Sombras Superior e Inferior (<i>Upper & Lower Shadows</i>)</b><br>
Quantificam a <b>rejeição de preços</b> em extremos intradiários.  
A <b>Sombra Superior</b> (<i>H<sub>t</sub> − max(O<sub>t</sub>, C<sub>t</sub>)</i>) indica pressão vendedora em níveis elevados, enquanto a <b>Sombra Inferior</b> (<i>min(O<sub>t</sub>, C<sub>t</sub>) − L<sub>t</sub></i>) reflete defesa compradora em níveis baixos.  
São vitais para detectar <b>exaustão de tendência</b> e <b>armadilhas de liquidez</b>.

</div>


In [ ]:
itau['Daily_Range'] = itau['High'] - itau['Low']
itau['Real_Body'] = itau['Close'] - itau['Open']
itau['Upper_Shadow'] = itau['High'] - itau[['Open', 'Close']].max(axis=1)
itau['Lower_Shadow'] = itau[['Open', 'Close']].min(axis=1) - itau['Low']
print(itau.head())

itau.columns

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Log-Retorno (<i>Log Returns</i>)</b><br>
Diferença entre o logaritmo natural do preço atual e o anterior (<i>ln(P<sub>t</sub> / P<sub>t−1</sub>)</i>).  
É preferível ao retorno simples por favorecer a <b>estacionariedade</b> da série e a <b>simetria estatística</b> — movimentos de alta e baixa de mesma intensidade possuem magnitudes numéricas equivalentes.  
Essa propriedade facilita a <b>convergência</b> e a estabilidade de algoritmos baseados em <b>gradient boosting</b>.

</div>


In [ ]:
itau['log_retorno'] = np.log(itau['Close']/itau['Close'].shift(1))
itau['Log_Retorno_t-1'] = itau['log_retorno'].shift(1)
print(itau['log_retorno'].head())
itau.columns

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Janelas Deslizantes — Médias e Volatilidade (Rolling Statistics)</b><br>
Estatísticas aplicadas sobre janelas de <b>14, 30 e 60 períodos</b>.  
A <b>Média Móvel</b> captura a tendência central de curto e médio prazo, enquanto a <b>Volatilidade</b> (desvio padrão dos <i>log-retornos</i>) quantifica o risco histórico recente.  
Essa dualidade permite ao modelo contextualizar o preço atual dentro de diferentes <b>regimes de mercado</b> — <i>calmo/direcional</i> versus <i>volátil/errático</i>.

</div>


In [ ]:
janelas = [14, 30, 60]

for n in janelas:
    itau[f'JanelaMedia{n}'] = itau['Close'].rolling(window=n).mean()

    itau[f'Volatilidade{n}'] = itau['log_retorno'].rolling(window=n).std() 

itau.columns

## II. Indicadores Técnicos e Momentum

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 10px;
">

<b>Média Móvel Simples (SMA — 7 e 21 períodos)</b><br>
Filtro linear que calcula a média aritmética dos preços de fechamento (<i>AdjClose</i>).  
Atua como um <b>suavizador de ruído de alta frequência</b>, permitindo ao modelo identificar a tendência prevalente.  
A janela de <b>7 períodos</b> captura o fluxo imediato (semanal), enquanto a de <b>21 períodos</b> representa o consenso mensal de valor.

</div>

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Média Móvel Exponencial (EMA — 12 e 26 períodos)</b><br>
Diferentemente da SMA, aplica um fator de ponderação recursivo que atribui maior peso aos dados mais recentes.  
É matematicamente mais <b>reativa</b> (<i>α = 2/(N + 1)</i>), permitindo detectar <b>reversões de tendência</b> com menor atraso (<i>lag</i>).  
Serve como base para indicadores derivados, sendo fundamental na construção do <b>MACD</b>.

</div>


In [ ]:
itau['SMA_7'] = itau['Close'].rolling(window=7).mean()
itau['SMA_21'] = itau['Close'].rolling(window=21).mean()

itau['EMA_12'] =itau['Close'].ewm(span=12, adjust=False).mean()
itau['EMA_26'] =itau['Close'].ewm(span=26, adjust=False).mean()

itau.columns

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>MACD (Moving Average Convergence Divergence)</b><br>
Oscilador calculado pela diferença entre a <b>EMA rápida (12)</b> e a <b>EMA lenta (26)</b>.  
Mede a <b>velocidade</b> e a <b>aceleração</b> da tendência.  
O modelo utiliza tanto a <b>linha MACD</b> quanto o seu <b>Sinal</b> (suavização de 9 períodos) para identificar <b>divergências</b> e o <b>momentum</b> do movimento atual em relação ao histórico.

</div>


In [ ]:

itau['MACD_Line'] = itau['EMA_12'] - itau['EMA_26']
itau['MACD_Signal'] = itau['MACD_Line'].ewm(span=9, adjust=False).mean()

itau.columns

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>RSI (Relative Strength Index — 7 e 14 períodos)</b><br>
Indicador de <b>momentum</b> normalizado entre 0 e 100, calculado pela razão suavizada (método de Wilder) entre ganhos e perdas médios.  
Identifica condições de <b>sobrecompra</b> (&gt; 70) e <b>sobrevenda</b> (&lt; 30), além da velocidade da mudança dos preços, sinalizando potenciais pontos de <b>reversão à média</b>.

</div>


In [ ]:

itau['Delta'] = itau['Close'].diff()
itau['Gain'] = itau['Delta'].clip(lower=0)
itau['Loss'] = itau['Delta'].clip(upper=0).abs()

periodos_rsi = [7, 14]

for n in periodos_rsi:
    avg_gain = itau['Gain'].ewm(alpha=1/n, min_periods=n, adjust=False).mean()
    avg_loss = itau['Loss'].ewm(alpha=1/n, min_periods=n, adjust=False).mean()
    
    rs = avg_gain / avg_loss
    
    itau[f'RSI_{n}'] = 100 - (100 / (1 + rs))
itau.drop(columns=['Delta', 'Gain', 'Loss'], inplace=True)

print(itau[['RSI_7', 'RSI_14']].tail())
itau.columns

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Bandas de Bollinger (21 períodos, 2 desvios)</b><br>
Envelope de volatilidade composto por uma média central e duas bandas externas (<i>μ ± 2σ</i>).  
Contextualizam o preço em termos estatísticos: toques nas bandas indicam desvios extremos da normalidade.  
Para o algoritmo, funcionam como medidas dinâmicas de <b>suporte</b>, <b>resistência</b> e <b>expansão de volatilidade</b>.

</div>


In [ ]:
itau['BBmean'] = itau['Close'].rolling(window=21).mean()
itau['BBstd'] = itau['Close'].rolling(window=21).std()

itau['BBupper'] = itau['BBmean'] + (2* itau['BBstd'])
itau['BBLower'] = itau['BBmean'] - (2* itau['BBstd'])

itau.columns

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>ATR (Average True Range)</b><br>
Mede a volatilidade absoluta considerando a amplitude do dia e os gaps de abertura em relação ao fechamento anterior.  
Essencial para modelagem de risco e definição de stops em pontos reais.

</div>


In [ ]:

itau['Prev_Close'] = itau['Close'].shift(1)

itau['Range_1'] = itau['High'] - itau['Low']
itau['Range_2'] = (itau['High'] - itau['Prev_Close']).abs()
itau['Range_3'] = (itau['Low'] - itau['Prev_Close']).abs()

itau['TR'] = itau[['Range_1', 'Range_2', 'Range_3']].max(axis=1)
itau['ATR_7'] = itau['TR'].rolling(window=7).mean()
itau['ATR_14'] = itau['TR'].rolling(window=14).mean()

itau.drop(columns=['Prev_Close', 'Range_1', 'Range_2', 'Range_3'], inplace=True)
print(itau[['TR', 'ATR_7', 'ATR_14']].tail())
itau.columns

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>SQZ (Squeeze Momentum)</b><br>
Indicador híbrido que identifica períodos de compressão de volatilidade.  
Matematicamente, ocorre quando as <b>Bandas de Bollinger</b> estreitam-se em relação aos <b>Canais de Keltner</b> (baseados em ATR).  
Sinaliza regimes de <b>acumulação silenciosa</b> que estatisticamente precedem movimentos explosivos de preço.

</div>


In [ ]:
itau['KC_Upper'] = itau['SMA_21'] + (1.5 * itau['ATR_14'])
itau['KC_Lower'] = itau['SMA_21'] - (1.5 * itau['ATR_14'])

condicao_teto = itau['BBupper'] < itau['KC_Upper']
condicao_chao = itau['BBLower'] > itau['KC_Lower']

itau['SQZ_On'] = (condicao_teto & condicao_chao).astype(int)
itau.drop(columns=['KC_Upper', 'KC_Lower'], inplace=True)

print(itau['SQZ_On'].value_counts())
itau.columns

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>KDJ (Oscilador Estocástico Derivado)</b><br>
Refinamento do oscilador estocástico clássico, composto pelas linhas <b>K</b> (rápida), <b>D</b> (lenta) e <b>J</b> (divergência).  
O KDJ localiza o fechamento atual dentro do intervalo Máxima–Mínima recente.  
A linha <b>J</b>, mais sensível, permite antecipar pontos de virada de curto prazo com maior precisão que o estocástico tradicional.

</div>


In [ ]:
itau['Low_14'] = itau['Low'].rolling(window=14).min()
itau['High_14'] = itau['High'].rolling(window=14).max()

itau['RSV'] = 100 * ((itau['Close'] - itau['Low_14']) / (itau['High_14'] - itau['Low_14']))

itau['K'] = itau['RSV'].ewm(com=2, adjust=False).mean()
itau['D'] = itau['K'].ewm(com=2, adjust=False).mean()
itau['J'] = (3 * itau['K']) - (2 * itau['D'])

itau.drop(columns=['Low_14', 'High_14', 'RSV'], inplace=True)

# Conferência final da Parte II
print("Colunas Finais da Parte II:")
print(itau.columns[-3:])

Dado que usaremos o XGboost, devemos aplicar um alvo, nesse sentido, criamos 2: Um alvo de retorno(Retorno 5 dias) e um alvo binário (Retorno positivo ou não em 5 dias)

In [ ]:
import numpy as np
import pandas as pd

itau_semanal = itau[itau.index.dayofweek == 4].copy()

itau_semanal.dropna(inplace=True)

itau_semanal['AlvoRetorno'] = itau_semanal['Close'].shift(-1) / itau_semanal['Close'] - 1
itau_semanal['Alvo'] = np.where(itau_semanal['AlvoRetorno'] > 0, 1, 0)

itau_semanal.dropna(subset=['Alvo', 'AlvoRetorno'], inplace=True)
itau_semanal['Alvo'] = itau_semanal['Alvo'].astype(int)

itau_semanal.index.to_series().to_csv('./dados/BaseA.csv', index=False, header=['Data'])

Seja o nosso rebalanceamento somente nas sextas feiras em um intervalo semanal, então filtramos somente os dados onde a data era sexta feira, a fim de evitar o ruido dos outros dias que só serviriam para prever 5D+, algo nao útil quando focamos na sexta.

Tratamento de feature

Coeficiente de Correlação de Pearson

In [ ]:
featuresTest = [c for c in itau_semanal.columns if c not in ['Alvo', 'AlvoRetorno', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
X = itau_semanal[featuresTest]
y = itau_semanal['Alvo']

corr_matrix = X.corr().abs() #pearsson 

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print(f"Features detectadas com alta correlação (> 0.95): {len(drop)}")
print(drop)

itau_final = X.drop(columns=drop)
print("Shape final das features após Pearson:", itau_final.shape)

In [ ]:
itau_final.columns

In [ ]:
mi_scores = mutual_info_classif(itau_final, y, random_state=42)
mi_series = pd.Series(mi_scores, index=itau_final.columns).sort_values(ascending=False)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(itau_final)
chi2_scores, p_values = chi2(X_scaled, y)
chi2_series = pd.Series(chi2_scores, index=itau_final.columns).sort_values(ascending=False)

print("TOP 10 Features por Informação Mútua")
print(mi_series.head(10))

print("\nTOP 10 Features por Qui-Quadrado")
print(chi2_series.head(10))

In [ ]:
lim = 0.01 

featSelect = chi2_series[chi2_series > lim].index.tolist()
featDrop = chi2_series[chi2_series <= lim].index.tolist()

itau_otimizado = itau_final[featSelect].copy()

print(f"Critério de Corte: Score > {lim}")
print(f"Total Original: {len(chi2_series)} variáveis")
print(f"Total Selecionado: {len(featSelect)} variáveis")
print("-" * 30)

print("\n VARIÁVEIS DESCARTADAS (Ruído):")
print(featDrop)

print("\n VARIÁVEIS MANTIDAS (Input do Modelo):")
print(itau_otimizado.columns.tolist())

print(itau_otimizado.describe().loc[['count', 'mean']])

In [ ]:
itau_otimizado.columns

Filtramos as variáveis com alta correlação via Pearson e validamos sua importância através dos testes de Qui-Quadrado e Informação Mútua. Observou-se que as métricas estão consistentemente distantes de zero (o que indicaria independência ou ruído), confirmando que estas variáveis possuem sinal suficiente para contribuir com o desempenho do modelo.

In [ ]:
print(itau_otimizado.isna().any())

In [ ]:
itau_otimizado.to_csv('./dados/teste.csv')

Automatizando para todas as ações

In [2]:
try:
    interseccao = pd.read_csv('./dados/Interseccao.csv')
    datasOfc = pd.to_datetime(interseccao['Data']).unique()
except:
    datasOfc = []

DATA_INICIAL = '2019-04-09' 
DATA_FIM = '2025-10-01'

ativos = pd.read_csv('ibovespa.csv', sep=';')

ativos.columns = [col.strip().lower() for col in ativos.columns]
tickers = ativos['ticker']

features_A = {}

print("Iniciando processamento")

for ticker in tickers:

    df = yf.download(ticker, start=DATA_INICIAL, end=DATA_FIM, progress=False, auto_adjust=True)
        
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    
    if df.empty:
        continue

    df['Daily_Range'] = df['High'] - df['Low']
    df['Real_Body'] = df['Close'] - df['Open']
    df['Upper_Shadow'] = df['High'] - df[['Open', 'Close']].max(axis=1)
    df['Lower_Shadow'] = df[['Open', 'Close']].min(axis=1) - df['Low']
    
    df['log_retorno'] = np.log(df['Close']/df['Close'].shift(1))
    df['Log_Retorno_t-1'] = df['log_retorno'].shift(1)

    janelas = [14, 30, 60]
    for n in janelas:
        df[f'JanelaMedia{n}'] = df['Close'].rolling(window=n).mean()
        df[f'Volatilidade{n}'] = df['log_retorno'].rolling(window=n).std() 

    df['SMA_7']  =  df['Close'].rolling(window=7).mean()
    df['SMA_21'] = df['Close'].rolling(window=21).mean()
    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()

    df['MACD_Line'] = df['EMA_12'] - df['EMA_26']
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()

    df['Delta'] = df['Close'].diff()
    df['Gain'] = df['Delta'].clip(lower=0)
    df['Loss'] = df['Delta'].clip(upper=0).abs()

    periodos_rsi = [7, 14]
    for n in periodos_rsi:
        avg_gain = df['Gain'].ewm(alpha=1/n, min_periods=n, adjust=False).mean()
        avg_loss = df['Loss'].ewm(alpha=1/n, min_periods=n, adjust=False).mean()
        rs = avg_gain / avg_loss
        df[f'RSI_{n}'] = 100 - (100 / (1 + rs))
    df.drop(columns=['Delta', 'Gain', 'Loss'], inplace=True)

    df['BBmean'] = df['Close'].rolling(window=21).mean()
    df['BBstd'] = df['Close'].rolling(window=21).std()
    df['BBupper'] = df['BBmean'] + (2* df['BBstd'])
    df['BBLower'] = df['BBmean'] - (2* df['BBstd'])

    df['Prev_Close'] = df['Close'].shift(1)
    df['Range_1'] = df['High'] - df['Low']
    df['Range_2'] = (df['High'] - df['Prev_Close']).abs()
    df['Range_3'] = (df['Low'] - df['Prev_Close']).abs()
    df['TR'] = df[['Range_1', 'Range_2', 'Range_3']].max(axis=1)
    df['ATR_7'] = df['TR'].rolling(window=7).mean()
    df['ATR_14'] = df['TR'].rolling(window=14).mean()
    df.drop(columns=['Prev_Close', 'Range_1', 'Range_2', 'Range_3'], inplace=True)
    
    df['KC_Upper'] = df['SMA_21'] + (1.5 * df['ATR_14'])
    df['KC_Lower'] = df['SMA_21'] - (1.5 * df['ATR_14'])
    condicao_teto = df['BBupper'] < df['KC_Upper']
    condicao_chao = df['BBLower'] > df['KC_Lower']
    df['SQZ_On'] = (condicao_teto & condicao_chao).astype(int)
    df.drop(columns=['KC_Upper', 'KC_Lower'], inplace=True)

    df['Low_14'] = df['Low'].rolling(window=14).min()
    df['High_14'] = df['High'].rolling(window=14).max()
    df['RSV'] = 100 * ((df['Close'] - df['Low_14']) / (df['High_14'] - df['Low_14']))
    df['K'] = df['RSV'].ewm(com=2, adjust=False).mean()
    df['D'] = df['K'].ewm(com=2, adjust=False).mean()
    df['J'] = (3 * df['K']) - (2 * df['D'])
    df.drop(columns=['Low_14', 'High_14', 'RSV'], inplace=True)

    df = df[df.index.dayofweek == 4].copy()

    df.dropna(inplace=True)

    df['AlvoRetorno'] = df['Close'].shift(-1) / df['Close'] - 1
    df['Alvo'] = np.where(df['AlvoRetorno'] > 0, 1, 0)
    
    df.dropna(subset=['Alvo', 'AlvoRetorno'], inplace=True)
    df['Alvo'] = df['Alvo'].astype(int)

    if len(datasOfc) > 0:
        df.index = pd.to_datetime(df.index)
        df = df[df.index.isin(datasOfc)].copy()

    featuresTest = [c for c in df.columns if c not in ['Alvo', 'AlvoRetorno', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    X = df[featuresTest]
    y = df['Alvo']

    if len(X) < 10:
        print(f"Aviso: {ticker} ficou com dados insuficientes. Pulando.")
        continue

    corr_matrix = X.corr().abs() 
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    var = X.drop(columns=drop)

    mi_scores = mutual_info_classif(var, y, random_state=42)
    mi_series = pd.Series(mi_scores, index=var.columns).sort_values(ascending=False)

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(var)
    chi2_scores, p_values = chi2(X_scaled, y)
    chi2_series = pd.Series(chi2_scores, index=var.columns).sort_values(ascending=False)

    lim = 0.01 
    featSelect = chi2_series[chi2_series > lim].index.tolist()
    
    varFinal = var[featSelect].copy()
    
    varFinal['Alvo'] = df['Alvo']
    varFinal['AlvoRetorno'] = df['AlvoRetorno']
    
    features_A[ticker] = varFinal

    print(f'{ticker} Processado')

print("Processo de features concluído")

Iniciando processamento
PETR4.SA Processado
ITUB4.SA Processado
VALE3.SA Processado
BPAC11.SA Processado
ABEV3.SA Processado
WEGE3.SA Processado
BBDC3.SA Processado
ITSA4.SA Processado
BBAS3.SA Processado
SANB11.SA Processado
VIVT3.SA Processado
SBSP3.SA Processado
BBSE3.SA Processado
B3SA3.SA Processado
SUZB3.SA Processado
CPFE3.SA Processado
TIMS3.SA Processado
EQTL3.SA Processado
RENT3.SA Processado
RADL3.SA Processado
NEOE3.SA Processado
ENEV3.SA Processado
GGBR3.SA Processado
EGIE3.SA Processado
CMIG4.SA Processado
PRIO3.SA Processado
VBBR3.SA Processado
RAIL3.SA Processado
TOTS3.SA Processado
ENGI11.SA Processado
KLBN11.SA Processado
UGPA3.SA Processado
CSAN3.SA Processado
CSMG3.SA Processado
GOAU4.SA Processado
HYPE3.SA Processado
ALOS3.SA Processado
MULT3.SA Processado
TAEE11.SA Processado
LREN3.SA Processado
REDE3.SA Processado
CSNA3.SA Processado
CYRE3.SA Processado
ALUP11.SA Processado
FLRY3.SA Processado
USIM5.SA Processado
BRAP4.SA Processado
HAPV3.SA Processado
SLCE3.SA P

Teste de datas iguais em ambos os modelos

In [ ]:
print(features_A['PETR4.SA'].tail(-10))

In [ ]:
print(features_A['PETR4.SA']['Alvo'])
print(features_A['PETR4.SA']['AlvoRetorno'])

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

modelosA = {}

for ticker, df in features_A.items():
    print(f"\n--- Treinando: {ticker} ---")
    
    features = [c for c in df.columns if c not in ['Alvo', 'AlvoRetorno']]
    X = df[features]
    y = df['Alvo']

    split = int(len(df) * 0.8)
    
    X_dev = X.iloc[:split]
    y_dev = y.iloc[:split]
    
    X_backtest = X.iloc[split:]
    y_backtest = y.iloc[split:]
    
    def objective(trial):

        cutoff = int(len(X_dev) * 0.75)
        X_train_opt, X_val_opt = X_dev.iloc[:cutoff], X_dev.iloc[cutoff:]
        y_train_opt, y_val_opt = y_dev.iloc[:cutoff], y_dev.iloc[cutoff:]
        
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
            'gamma': trial.suggest_float('gamma', 0.0, 5.0),
            'random_state': 42,
            'n_jobs': -1
        }
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_train_opt, y_train_opt)
        
        preds = model.predict(X_val_opt)
        
        score = precision_score(y_val_opt, preds, zero_division=0)
        
        return score

    study = optuna.create_study(direction='maximize')
    
    study.optimize(objective, n_trials=100) 
    
    print(f"Melhor Score: {study.best_value:.2%}")
    print(f"Melhores Parâmetros: {study.best_params}")

    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['n_jobs'] = -1
    
    final_model = xgb.XGBClassifier(**best_params)
    final_model.fit(X_dev, y_dev)
    
    modelosA[ticker] = final_model

In [ ]:

joblib.dump(modelosA, '../modelos/ModelosA.pkl')
print("Arquivo mestre salvo com sucesso.")

In [3]:
treino = {}

for ticker, df in features_A.items():

    split = int(len(df) * 0.8)
    
    dfcut = df.iloc[split:].copy()
    
    treino[ticker] = dfcut

joblib.dump(treino, '../FeaturesA.pkl')

['../FeaturesA.pkl']